In [1]:
import os

# Directorio con los archivos del corpus
corpus_dir = "cornell movie-dialogs corpus"
lines_file = os.path.join(corpus_dir, "movie_lines.txt")
conversations_file = os.path.join(corpus_dir, "movie_conversations.txt")

In [ ]:
# Cada línea tiene 5 campos separados por " +++$+++ "
id2line = {}
with open(lines_file, encoding='utf-8', errors='ignore') as file:
    for line in file:
        parts = line.strip().split(" +++$+++ ")
        if len(parts) == 5:
            line_id = parts[0]
            text = parts[4]
            id2line[line_id] = text

In [3]:
pairs = []
with open(conversations_file, encoding='utf-8', errors='ignore') as file:
    for line in file:
        parts = line.strip().split(" +++$+++ ")
        if len(parts) == 4:
            utterance_ids = eval(parts[3])  # convierte lista de IDs en lista de Python
            for i in range(len(utterance_ids) - 1):
                q = id2line.get(utterance_ids[i])
                a = id2line.get(utterance_ids[i+1])
                if q and a:
                    pairs.append((q.strip(), a.strip()))

In [7]:
import pandas as pd

# Crear el DataFrame
df_chat = pd.DataFrame(pairs, columns=["pregunta", "respuesta"])

# Mostrar las primeras filas
print(df_chat.head())

                                            pregunta  \
0  Can we make this quick?  Roxanne Korrine and A...   
1  Well, I thought we'd start with pronunciation,...   
2  Not the hacking and gagging and spitting part....   
3  You're asking me out.  That's so cute. What's ...   
4  No, no, it's my fault -- we didn't have a prop...   

                                           respuesta  
0  Well, I thought we'd start with pronunciation,...  
1  Not the hacking and gagging and spitting part....  
2  Okay... then how 'bout we try out some French ...  
3                                         Forget it.  
4                                           Cameron.  


In [8]:
df_chat.shape

(221282, 2)

In [10]:
df_chat.to_csv("dataset_chatbot.csv", index=False)